- For resonant and Ari TTV systems use REBOUND commit dd5dc543f5138b4ab32471a26230d39f8f2fcd7c
- For random and Naireen systems use REBOUND commit 48feb327f90611a5569682578980b5604aa6102a on the oldMLstability branch

The resonant and Ari TTV systems use different REBOUND commits (a675e6f92cba32bae86f94739faec3b1b3098e81 for resonant). The problem is that this version doesn't have a __del__ function for the simulation archives and you get an error of too many open files when you run this. There was also a bug with keep_unsynchronized=1 and using MEGNO, so we now use the patch dd5dc543f5138b4ab32471a26230d39f8f2fcd7c. I test below that an integration that initializes MEGNO gives the same result as the original sim archives that don't.

The same commit 48feb327f90611a5569682578980b5604aa6102a works for the random and Naireen distribution systems. I had to branch from the old commit used to run the long integrations db3ae2cea8f3462463d3e0c5788a34625bb49a9c (actually it's not the exact one but close to it), and add the patches to calculate megno/lyap correctly with keep_unsynchronized=1 (which we need to match bit by bit). 

Depending on which you load above, that pair will give the same results below, the other two will not (or won't work). But the same testsim function can work on all which lets us always use the same function in generatedata/training_data_functions.py. 

When we ran the resonant integrations (with the new simarchive), we did with safe_mode = 0 so we don't repeat kepler half steps. This means the sim will be in an unsynchronized state (see simarchiverestart ipython_example) when we load any snapshot but the first one, so we have to synchronize before comparing output. 

We need to use sim.ri_whfast.keep_unsynchronized = 1 for reproducibility. A value of 1 means that even if you synchronize, you'll keep integrating from the unsynchronized state if you keep going. So when integrating through a list of times, we need to set this if we want to match the sim archives. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd
import rebound
%matplotlib inline

In [2]:
def collision(reb_sim, col):
    reb_sim.contents._status = 5
    return 0

def testsim(sim, times):
    sim.collision_resolve = collision
    sim.ri_whfast.keep_unsynchronized = 1
    sim.init_megno()
    val = np.zeros((Nout, 19))
    for i, time in enumerate(times):
        sim.integrate(time, exact_finish_time=0)
        orbits = sim.calculate_orbits()
        for j, o in enumerate(orbits):
            val[i,0] = sim.t
            val[i,6*j+1] = o.a
            val[i,6*j+2] = o.e
            val[i,6*j+3] = o.inc
            val[i,6*j+4] = o.Omega
            val[i,6*j+5] = o.pomega
            val[i,6*j+6] = o.M
    return sim, val

In [3]:
Nout = 1729

# Random

In [26]:
sa = rebound.SimulationArchive('/mnt/ssd/workspace/stability/stabilitydataset/data/random/simulation_archives/runs/sa0000000.bin')
tmax = sa[1].t
sim = sa[0]

times = np.linspace(0, tmax, Nout)
sim, val = testsim(sim, times) # one of the data generation functions we run over short period
sim.status()

---------------------------------
REBOUND version:     	3.4.0
REBOUND built on:    	Apr  5 2019 22:55:24
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	1.0000000277011167e+06
Current timestep:    	0.034640
---------------------------------
<rebound.Particle object, m=1.0 x=-3.629182535523935e-05 y=1.1843522755585877e-05 z=9.692032962716541e-08 vx=-2.7539975849184616e-05 vy=-0.00021744008363471437 vz=-8.43155749795529e-07>
<rebound.Particle object, m=3.4140160192072926e-05 x=0.9225193262954728 y=0.4107034942938748 z=-0.007950327673330494 vx=-2.4266312875245175 vy=5.726893721352419 vz=0.04678499897728613>
<rebound.Particle object, m=1.8787249753397036e-05 x=0.16981871403865156 y=-1.2279135896541034 z=0.011612920399274694 vx=5.527494030376944 vy=0.9037972044771627 vz=-0.06443832825556559>
<rebound.Particle object, m=1.8268998599127623e-06 x=0.87932635067516 y=-1.5303994574427984 z=-0.02390372135108166 vx=3.5793674392077772 vy=2.705759021771257 vz=0.249891834123

In [27]:
sim = sa[1]
sim.integrator_synchronize()
sim.status()

---------------------------------
REBOUND version:     	3.4.0
REBOUND built on:    	Apr  5 2019 22:55:24
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	1.0000000277011167e+06
Current timestep:    	0.034640
---------------------------------
<rebound.Particle object, m=1.0 x=-3.629182535523935e-05 y=1.1843522755585877e-05 z=9.692032962716541e-08 vx=-2.7539975849184616e-05 vy=-0.00021744008363471437 vz=-8.43155749795529e-07>
<rebound.Particle object, m=3.4140160192072926e-05 x=0.9225193262954728 y=0.4107034942938748 z=-0.007950327673330494 vx=-2.4266312875245175 vy=5.726893721352419 vz=0.04678499897728613>
<rebound.Particle object, m=1.8787249753397036e-05 x=0.16981871403865156 y=-1.2279135896541034 z=0.011612920399274694 vx=5.527494030376944 vy=0.9037972044771627 vz=-0.06443832825556559>
<rebound.Particle object, m=1.8268998599127623e-06 x=0.87932635067516 y=-1.5303994574427984 z=-0.02390372135108166 vx=3.5793674392077772 vy=2.705759021771257 vz=0.249891834123

# Naireen

In [24]:
sa = rebound.SimulationArchive('/mnt/ssd/workspace/stability/naireenlustre/data/distributions/Sys_1_1e8/simulation_archives/sa0000000.bin')
tmax = sa[1].t
sim = sa[0]
times = np.linspace(0, tmax, Nout)
sim, val = testsim(sim, times) # one of the data generation functions we run over short period
sim.status()

---------------------------------
REBOUND version:     	3.4.0
REBOUND built on:    	Apr  5 2019 22:55:24
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	1.0000000999712122e+05
Current timestep:    	0.034641
---------------------------------
<rebound.Particle object, m=1.0 x=-9.173480652651816e-06 y=-4.354427286039064e-05 z=5.542083599589504e-07 vx=0.00019654769538463409 vy=2.6979549413033457e-05 vz=4.954382605171793e-06>
<rebound.Particle object, m=2.5298381201802596e-07 x=0.8470725506233061 y=0.41095563040632466 z=0.03976918629748858 vx=-3.197193747693299 vy=5.8395491651175195 vz=-0.03451638541126706>
<rebound.Particle object, m=3.485789928918549e-05 x=-0.6897739638334781 y=0.9504282798666671 z=-0.005880691556774625 vx=-4.833846067332019 vy=-3.2673866283777704 vz=-0.19088740917165398>
<rebound.Particle object, m=1.9557973661340233e-05 x=1.6874578595236527 y=0.5271698723569521 z=-0.01837004039421574 vx=-1.392840495636032 vy=4.368416416186001 vz=0.087344609193

In [25]:
sim = sa[1]
sim.integrator_synchronize() # need this step with new sim archive because we used safe_mode = 0
sim.status()

---------------------------------
REBOUND version:     	3.4.0
REBOUND built on:    	Apr  5 2019 22:55:24
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	1.0000000999712122e+05
Current timestep:    	0.034641
---------------------------------
<rebound.Particle object, m=1.0 x=-9.173480652651816e-06 y=-4.354427286039064e-05 z=5.542083599589504e-07 vx=0.00019654769538463409 vy=2.6979549413033457e-05 vz=4.954382605171793e-06>
<rebound.Particle object, m=2.5298381201802596e-07 x=0.8470725506233061 y=0.41095563040632466 z=0.03976918629748858 vx=-3.197193747693299 vy=5.8395491651175195 vz=-0.03451638541126706>
<rebound.Particle object, m=3.485789928918549e-05 x=-0.6897739638334781 y=0.9504282798666671 z=-0.005880691556774625 vx=-4.833846067332019 vy=-3.2673866283777704 vz=-0.19088740917165398>
<rebound.Particle object, m=1.9557973661340233e-05 x=1.6874578595236527 y=0.5271698723569521 z=-0.01837004039421574 vx=-1.392840495636032 vy=4.368416416186001 vz=0.087344609193

# Resonant

In [4]:
sa = rebound.SimulationArchive('/mnt/ssd/workspace/stability/stabilitydataset/data/resonant/simulation_archives/runs/sa0000000.bin')
tmax = sa[1].t
sim = sa[0]
times = np.linspace(0, tmax, Nout)
sim, val = testsim(sim, times) # one of the data generation functions we run over short period
sim.status()

/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)


---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	Apr  5 2019 23:19:09
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	1.0000000173176938e+06
Current timestep:    	0.034641
---------------------------------
<rebound.Particle object, m=0.9999960150799597 x=-3.658542143467641e-06 y=-1.949623323935794e-05 z=9.470806349629878e-08 vx=6.872767724455177e-05 vy=-3.73442855666929e-06 vz=3.496108378488267e-07>
<rebound.Particle object, m=3.984920040268468e-06 x=-0.8738958364059268 y=0.4817517647367535 z=-0.003159573737010137 vx=-2.9307195320280233 vy=-5.572709926922893 vz=0.0012280114770667806>
<rebound.Particle object, m=1.2444171101394952e-05 x=0.6063431581766432 y=1.4011530424916374 z=-0.007375445763490789 vx=-4.553768175624738 vy=2.174789434522493 vz=-0.026555396336484776>
<rebound.Particle object, m=2.531702420746453e-07 x=-1.5977605108526833 y=0.5538766686714539 z=0.03817301570906049 vx=-1.5043238809838 vy=-4.4327274873988936 vz=

In [5]:
sim = sa[1]
sim.integrator_synchronize()
sim.status()

---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	Apr  5 2019 23:19:09
Number of particles: 	4
Selected integrator: 	whfast
Simulation time:     	1.0000000173176938e+06
Current timestep:    	0.034641
---------------------------------
<rebound.Particle object, m=0.9999960150799597 x=-3.658542143467641e-06 y=-1.949623323935794e-05 z=9.470806349629878e-08 vx=6.872767724455177e-05 vy=-3.73442855666929e-06 vz=3.496108378488267e-07>
<rebound.Particle object, m=3.984920040268468e-06 x=-0.8738958364059268 y=0.4817517647367535 z=-0.003159573737010137 vx=-2.9307195320280233 vy=-5.572709926922893 vz=0.0012280114770667806>
<rebound.Particle object, m=1.2444171101394952e-05 x=0.6063431581766432 y=1.4011530424916374 z=-0.007375445763490789 vx=-4.553768175624738 vy=2.174789434522493 vz=-0.026555396336484776>
<rebound.Particle object, m=2.531702420746453e-07 x=-1.5977605108526833 y=0.5538766686714539 z=0.03817301570906049 vx=-1.5043238809838 vy=-4.4327274873988936 vz=

# Ari

In [9]:
sa = rebound.SimulationArchive('/mnt/ssd/workspace/stability/stabilitydataset/data/nonressystems/LP-358-499/simulation_archives/runs/sa0000000.bin')

# The innermost planet now has a different orbital period, but the timestep is still the same fraction to close to machine precision:

In [10]:
sim = sa[0]
sim.dt/sim.particles[1].P

/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


0.034641016151377546

In [11]:
tmax = sa[1].t
sim = sa[0]
times = np.linspace(0, tmax, Nout)
sim, val = testsim(sim, times) # one of the data generation functions we run over short period
sim.status()

---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	Apr  5 2019 23:19:09
Number of particles: 	8
Selected integrator: 	whfast
Simulation time:     	5.2867929037495232e+04
Current timestep:    	0.001831
---------------------------------
<rebound.Particle object, m=0.51 x=-9.243438619388608e-06 y=-5.843309887933348e-06 z=-5.364867056784226e-09 vx=0.00025812578933250956 vy=-0.0004070578950174552 vz=3.717095432586538e-07>
<rebound.Particle object, m=6.606794029325489e-06 x=0.029415138260382036 y=-0.011719099624968315 z=-2.0293887473444915e-05 vx=1.555005645878225 vy=3.789031687122423 vz=-0.000406892740939987>
<rebound.Particle object, m=2.110761752620521e-05 x=0.018112352912284243 y=0.04141347641253959 z=2.7088584243370827e-05 vx=-3.0000505678412064 vy=1.485960978933363 vz=-0.004054656657424565>
<rebound.Particle object, m=6.33670720508826e-05 x=0.0652942480233223 y=0.034455969465934946 z=3.627095560720546e-05 vx=-1.2402945581572582 vy=2.3861143277678045 vz=

In [12]:
sim = sa[1]
sim.integrator_synchronize() # need this step with new sim archive because we used safe_mode = 0
sim.status()

---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	Apr  5 2019 23:19:09
Number of particles: 	4
Selected integrator: 	whfast
Simulation time:     	5.2867929037495232e+04
Current timestep:    	0.001831
---------------------------------
<rebound.Particle object, m=0.51 x=-9.243438619388608e-06 y=-5.843309887933348e-06 z=-5.364867056784226e-09 vx=0.00025812578933250956 vy=-0.0004070578950174552 vz=3.717095432586538e-07>
<rebound.Particle object, m=6.606794029325489e-06 x=0.029415138260382036 y=-0.011719099624968315 z=-2.0293887473444915e-05 vx=1.555005645878225 vy=3.789031687122423 vz=-0.000406892740939987>
<rebound.Particle object, m=2.110761752620521e-05 x=0.018112352912284243 y=0.04141347641253959 z=2.7088584243370827e-05 vx=-3.0000505678412064 vy=1.485960978933363 vz=-0.004054656657424565>
<rebound.Particle object, m=6.33670720508826e-05 x=0.0652942480233223 y=0.034455969465934946 z=3.627095560720546e-05 vx=-1.2402945581572582 vy=2.3861143277678045 vz=

# Test to show that even though I'm not reproducible when using newest REBOUND on old (random) integrations, I can still load the binary and get the latest time and masses, so I can always use same commit for generate_data/generate_metadata.py


In [13]:
sim = rebound.Simulation('/mnt/ssd/workspace/stability/stabilitydataset/data/random/final_conditions/runs/fc0000000.bin')
sim.t

/mnt/ssd/workspace/rebound/rebound/simulation.py:352: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulation.py:352: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulation.py:352: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)


12356442.121674882

In [14]:
sa = rebound.SimulationArchive('/mnt/ssd/workspace/stability/stabilitydataset/data/random/simulation_archives/runs/sa0000000.bin')
sim = sa[0]

In [15]:
sim.particles[1].m

3.4140160192072926e-05